In [87]:
from postgres_connect import *
import numpy as np


##### Revenue

In [88]:
#change below for date----------------------------------------------------------------
query = '''
        select awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city from public.revenue_24_jun 
        where attributed_month = '2024-03-01'       
'''
df_revenue = get_df_from_sql('Yes',query,'tunnel-ssh.cer')
df_revenue.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
0,GS1205975883,Decathlon,Bangalore,560100,Delivered,Prepaid,60.0,54.05,301,Bangalore
1,GS2131162508,Shiprocket NDD,Bangalore,560099,RTO Delivered,COD,300.0,34.50,301,Bangalore
2,GS1123757704,Shiprocket Intercity,Bangalore,500089,Delivered,Prepaid,392.0,31.00,301,Hyderabad
3,GS1443308699,adidas,Delhi,110045,Delivered,Prepaid,500.0,21.00,301,Delhi
4,GS1788247667,Bebodywise,Bangalore,560011,RTO Delivered,COD,180.0,36.00,301,Bangalore


In [89]:
df_revenue_filtered = df_revenue[(df_revenue['shipper_id'] == 301) & (df_revenue['shipping_city'] != 'Pune')]
df_revenue_filtered['warehouse_city'] = df_revenue_filtered['warehouse_city'].replace('NCR', 'Delhi')
df_revenue_filtered['shipping_city'] = df_revenue_filtered['warehouse_city'].replace('NCR', 'Delhi')
df_revenue_filtered.groupby('warehouse_city').aggregate('count')

,awb,user_name,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
warehouse_city,,,,,,,,,
Bangalore,192720,192720,192720,192720,192720,192720,192720,192720,192720
Delhi,140233,140233,140233,140233,140233,140233,140233,140233,140233
Hyderabad,37053,37053,37053,37053,37053,37053,37053,37053,37053
Mumbai,76130,76130,76130,76130,76130,76130,76130,76130,76130


In [90]:
df_revenue_filtered.shape

(446136, 10)

In [91]:
duplicate_revenue = df_revenue_filtered[df_revenue_filtered.duplicated('awb', keep=False)]
duplicate_revenue.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city


In [92]:
df_revenue_filtered.groupby('shipping_city').aggregate('count')

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id
shipping_city,,,,,,,,,
Bangalore,192720,192720,192720,192720,192720,192720,192720,192720,192720
Delhi,140233,140233,140233,140233,140233,140233,140233,140233,140233
Hyderabad,37053,37053,37053,37053,37053,37053,37053,37053,37053
Mumbai,76130,76130,76130,76130,76130,76130,76130,76130,76130


In [93]:
print('Delivered shape',df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered'].shape)
print('RTO Delivered shape',df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered'].shape)
print('Delivered Revenue',df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered']['shipment_charges'].sum())
print('RTO Delivered Revenue',df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered']['shipment_charges'].sum())

Delivered shape (416541, 10)
RTO Delivered shape (29595, 10)
Delivered Revenue 26340992.50558864
RTO Delivered Revenue 2688991.4562999997


In [94]:
df_revenue_filtered['shipment_charges'].sum()


np.float64(29029983.961888645)

In [95]:
df_revenue_delivered = df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered']
df_revenue_rto = df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered']

In [96]:
df_del_grouped = df_revenue_delivered.groupby(['shipping_city','shipment_status']).aggregate({
    'awb':'count'
}).reset_index().rename(columns={'awb':'orders'})
df_del_grouped.head()

,shipping_city,shipment_status,orders
0,Bangalore,Delivered,183543
1,Delhi,Delivered,127657
2,Hyderabad,Delivered,34688
3,Mumbai,Delivered,70653


In [97]:
df_rto_grouped = df_revenue_rto.groupby(['shipping_city','shipment_status']).aggregate({
    'awb':'count'
}).reset_index().rename(columns={'awb':'orders'})
df_rto_grouped.head()

,shipping_city,shipment_status,orders
0,Bangalore,RTO Delivered,9177
1,Delhi,RTO Delivered,12576
2,Hyderabad,RTO Delivered,2365
3,Mumbai,RTO Delivered,5477


In [98]:
df_rto_per = df_del_grouped.merge(df_rto_grouped,on='shipping_city',how='left')
df_rto_per['rto_per']=df_rto_per['orders_y']/(df_rto_per['orders_x']+df_rto_per['orders_y'])
df_rto_per.head()

,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per
0,Bangalore,Delivered,183543,RTO Delivered,9177,0.047618
1,Delhi,Delivered,127657,RTO Delivered,12576,0.089679
2,Hyderabad,Delivered,34688,RTO Delivered,2365,0.063827
3,Mumbai,Delivered,70653,RTO Delivered,5477,0.071943


##### Payout

In [99]:
df_rider_cost = pd.read_csv('Rider Payout Sheet All Months - 6_month (7).csv')
df_rider_cost = df_rider_cost.drop(columns=['PFESIC','Finder Fees','Commission','Vendor Pay','MIS PAY'])
df_rider_cost.head()    

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,exist
0,Bangalore,3637.0,chandu M,DS BLR HBBL,3WH,4 Wheeler,11080.0,0,0.0,0.0,0,11080.0,2024-01-01,Bangalore-45292-3WH,NaN
1,Bangalore,2053.0,DILIP KUMAR TATA ACE,DS BLR MTH,3WH,FIXED,14680.0,1295,0.0,0.0,1295,13385.0,2024-01-01,Bangalore-45292-3WH,NaN
2,Bangalore,2733.0,Manjunath R,DS BLR BOMM,3WH,FIXED,16775.0,0,0.0,0.0,0,16775.0,2024-01-01,Bangalore-45292-3WH,NaN
3,Bangalore,5284.0,Chethu,DS BLR BOMM,3WH,FIXED,4265.0,0,0.0,0.0,0,4265.0,2024-01-01,Bangalore-45292-3WH,NaN
4,Bangalore,2450.0,M Chandu,DS BLR HBBL,3WH,ORDER,5264.0,0,1514.0,0.0,1514,3750.0,2024-01-01,Bangalore-45292-3WH,NaN


In [100]:
#change below for date----------------------------------------------------------------
df_rider_cost_filtered = df_rider_cost[df_rider_cost['date']=='2024-03-01']
df_rider_cost_filtered['rider_id']=df_rider_cost_filtered['rider_id'].astype(int)


##### Vendor

In [101]:
df_vendor_cost = pd.read_csv('Vendor Pay Master - March-Rider-Vendor.csv')
df_vendor_cost.head()

,Key,City,Month,Vendor,Total Pay,Net Deduction,Net Pay,Negative Pay,PFESI,Sourcing Fees,Commission,Comm. Amount,Vendor Pay,MIS PAY,MIS Amount,Dif In MIS,Diff in Invoice,Extra per rs
0,Bangalore-45352-3WH,Bangalore,01/03/2024,3WH,37330.0,0.00,37330.00,₹0,0.0,0.0,NaN,0.0000,37330.000,37330.000,0.00,"-₹37,330",NaN,0.000000
1,Bangalore-45352-Blitz_Self,Bangalore,01/03/2024,Blitz_Self,73010.0,8748.50,64261.50,₹0,0.0,0.0,NaN,0.0000,854.000,9602.500,0.00,"-₹9,603",NaN,0.000000
2,Bangalore-45352-Hindusree,Bangalore,01/03/2024,Hindusree,1484495.0,46630.83,1437864.17,₹0,0.0,0.0,5%,71893.2085,1509757.379,1556388.209,1577378.25,"₹20,990",NaN,0.048429
3,Bangalore-45352-Olive,Bangalore,01/03/2024,Olive,44230.0,0.00,44230.00,₹0,NaN,0.0,5%,2473.0000,44230.000,44230.000,46441.50,"₹2,212",NaN,0.055912
4,Bangalore-45352-Peel works,Bangalore,01/03/2024,Peel works,1443740.0,182608.09,1261131.91,"₹23,244",114239.0,24000.0,6%,90050.0000,1512664.780,1695272.870,1539771.90,"-₹155,501",NaN,0.174223


##### FM-MM

In [102]:
df_mm_cost = pd.read_csv('Vendor Pay Master - FM-MM.csv')
df_mm_cost.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
0,01/01/2024,Bangalore,Bangalore,NaN,"₹11,18,555",NaN,Hyperlocal
1,01/01/2024,Delhi,Bangalore,NaN,"₹2,05,699",NaN,Intercity
2,01/01/2024,Delhi,Delhi,NaN,"₹12,91,611",NaN,Hyperlocal
3,01/01/2024,Bangalore,Hyderabad,Rippler,"₹2,94,664",NaN,Intercity
4,01/01/2024,Bangalore,Hyderabad,General Travels,"₹1,03,040",NaN,Intercity


In [103]:
df_mm_cost_filtered = df_mm_cost[df_mm_cost['Month']=='01/03/2024']
def convert_currency_to_number(currency_str):
    if pd.isna(currency_str):
        return None
    return int(currency_str.replace('₹', '').replace(',', ''))

# Apply the function to the Cost column
df_mm_cost_filtered['Cost'] = df_mm_cost_filtered['Cost'].apply(convert_currency_to_number).astype(float)
df_mm_cost_filtered.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
18,01/03/2024,Delhi,Bangalore,Rippler,626997.0,NaN,Intercity
19,01/03/2024,Bangalore,Bangalore,NaN,1657356.0,NaN,Hyperlocal
20,01/03/2024,Delhi,Delhi,NaN,1412767.0,NaN,Hyperlocal
21,01/03/2024,Delhi,Hyderabad,Rippler,334523.0,NaN,Intercity
22,01/03/2024,Bangalore,Hyderabad,Rippler,665012.0,NaN,Intercity


##### Fuel

In [104]:
df_fuel_cost = pd.read_csv('Vendor Pay Master - Fuel.csv')
df_fuel_cost.head()

,city,amount,Month
0,Delhi,565626,01/01/2024
1,Bangalore,599032,01/01/2024
2,Hyderabad,352854,01/01/2024
3,Mumbai,379773,01/01/2024
4,Delhi,347268,01/02/2024


In [105]:
df_fuel_cost_filtered = df_fuel_cost[df_fuel_cost['Month']=='01/03/2024']
df_fuel_cost_filtered.head()

,city,amount,Month
18,Delhi,414632,01/03/2024
19,Jaipur,105902,01/03/2024
20,Bangalore,209872,01/03/2024
21,Bangalore,126637,01/03/2024
22,Bangalore,211180,01/03/2024


In [106]:
df_fuel_cost_grouped=df_fuel_cost_filtered.groupby('city').aggregate({'amount':'sum'}).reset_index()
df_fuel_cost_grouped.head()

,city,amount
0,Bangalore,547689
1,Delhi,414632
2,Hyderabad,477601
3,Jaipur,105902
4,Mumbai,509398


##### Lost on Hub

In [107]:
df_hubloss_cost = pd.read_csv('Vendor Pay Master - lostonhub (1).csv')
df_hubloss_cost.head()

,Head,Team,wh_code,Amount,Month
0,Loss on Hub,Bangalore,DS_BLR_MTH,"64,669",01/01/2024
1,Loss on Hub,Mumbai,DS_BOM_KRL,"35,581",01/01/2024
2,Loss on Hub,Hyderabad,DS_HYD_KMP,"37,712",01/01/2024
3,Loss on Hub,Delhi,DS_NCR_DWK,"76,844",01/01/2024
4,Loss on Hub,Bangalore,DS_BLR_MTH,"69,881",01/02/2024


##### Franchise

In [108]:
df_franchise_cost = pd.read_csv('Rider Payout Sheet All Months - Fran_6_month.csv')
df_franchise_cost.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
0,2024-01-01,Bangalore,BLR FRH ECTY,"556,840",0,"18,947",0,"18,947","537,893"
1,2024-01-01,Bangalore,CMRJ-Franchise,"63,640","8,974","14,798",0,"23,771","39,869"
2,2024-01-01,Bangalore,STNG-Franchise,"11,600",0,"1,910",0,"1,910","9,690"
3,2024-01-01,Bangalore,UUTR-Franchise,"325,120",198,"23,478",0,"23,676","301,444"
4,2024-01-01,Delhi,DEL FRH GKP,"667,102","13,645","64,470",0,"78,115","588,987"


In [109]:
df_franchise_cost_filtered = df_franchise_cost[df_franchise_cost['date']=='2024-03-01']
df_franchise_cost_filtered.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
25,2024-03-01,Bangalore,BLDR-Franchise,"405,750",0,999,0,999,"404,751"
26,2024-03-01,Bangalore,BLR FRH ECTY,"621,270",0,578,0,578,"620,692"
27,2024-03-01,Bangalore,CMRJ-Franchise,"271,202",0,"2,547",0,"2,547","268,655"
28,2024-03-01,Bangalore,STNG-Franchise,"177,328",0,568,0,568,"176,760"
29,2024-03-01,Bangalore,UUTR-Franchise,"395,844","47,031","4,492",0,"51,523","344,321"


##### Delivered

In [110]:
query_kaptaan = ''' 
    with locus as (
    select * from locus_task_brief where date_trunc('month',dispatch_time)::date = '2024-05-01'
    )
    
    select 
        COALESCE(tour.rider_id, r.rider_id) AS num_rider_id
    ,    COALESCE(t.trip_name, l.awb) as awb 
    ,   t.trip_id
    ,   coalesce(t.start_at,l.dispatch_time) as dispatch_time
    ,   coalesce(n.node_name,nl.node_name) as node_name
    ,   coalesce(n.city_name ,nl.city_name) as shipping_city
    from application_db.trip t
    left join locus as l on l.task_id = t.locus_trip_id
    left join application_db.tour on t.tour_id = tour.tour_id
    left join application_db.node n on tour.node_id = n.node_id
    left join application_db.rider as r on l.rider_id = r.locus_rider_id
    left join application_db.node nl on l.location_id = nl.locus_home_base_id

    where (t.status = 'COMPLETED' or l.status = 'COMPLETED')
    and date_trunc('month',t.start_at)::date = '2024-03-01'
'''

df_delivered_awb = get_df_from_sql('Yes',query_kaptaan,'tunnel-ssh.cer')

df_delivered_awb.head()

,num_rider_id,awb,trip_id,dispatch_time,node_name,shipping_city
0,4030,GS2096598829,3375264,2024-03-19 07:41:07.843,DS BOM THAN,Mumbai
1,4030,GS1999554765,3375265,2024-03-19 07:41:03.574,DS BOM THAN,Mumbai
2,4030,GS1810417971,3375204,2024-03-19 07:41:18.123,DS BOM THAN,Mumbai
3,4030,GS1446543372,3375206,2024-03-19 07:40:59.170,DS BOM THAN,Mumbai
4,4030,GS1824091251,3375202,2024-03-19 07:40:53.029,DS BOM THAN,Mumbai


In [111]:
duplicate_awb = df_delivered_awb[df_delivered_awb.duplicated('awb', keep=False)]
print(duplicate_awb.shape)
print(duplicate_awb)

(98, 6)
        num_rider_id            awb  trip_id           dispatch_time  \
13638           4581   GS1549195994  3056965 2024-03-01 04:46:32.908   
14532           1000   GS1549195994  3057362 2024-03-01 04:46:56.593   
17806           2698  BZPPL00175529  3068817 2024-03-02 04:22:31.296   
17971           3970  BZPPL00175529  3068653 2024-03-02 04:26:46.436   
33012           1972   GS1311783926  3085831 2024-03-03 04:28:25.920   
...              ...            ...      ...                     ...   
401101          5069   GS1732780774  3556281 2024-03-30 06:11:53.671   
401102          5069   GS1563931380  3556282 2024-03-30 06:11:59.660   
401103          5069   GS1443499145  3556280 2024-03-30 06:12:11.184   
407079          3690   GS1664082496  3560560 2024-03-31 04:21:46.006   
407113          3690   GS1664082496  3560311 2024-03-31 04:07:19.419   

             node_name shipping_city  
13638   BLDR-Franchise     Bangalore  
14532   BLDR-Franchise     Bangalore  
17806   UU

In [112]:
# handling duplicate rows
df_delivered_awb['dispatch_time'] = pd.to_datetime(df_delivered_awb['dispatch_time'])
idx = df_delivered_awb.groupby('awb')['dispatch_time'].idxmax()
df_delivered_awb = df_delivered_awb.loc[idx]

In [113]:
# df_delivered_awb['node_name'].replace({'FBD-Franchise':'FABD-Franchise'},inplace=True)
# df_delivered_awb['node_name'].replace({'DS DEL HKMN':'KTWS-Franchise'},inplace=True)
# df_delivered_awb['num_rider_id'].replace({'5473':'6950'},inplace=True)
rider_ids_to_update = [5050, 5051, 5073, 5074, 5080, 5093, 5094]
df_delivered_awb.loc[df_delivered_awb['num_rider_id'].isin(rider_ids_to_update), 'node_name'] = 'BRBN-Franchise'
rider_ids_to_update_to_bldr = [2192,3376,4610]
df_delivered_awb.loc[df_delivered_awb['num_rider_id'].isin(rider_ids_to_update_to_bldr), 'node_name'] = 'BLDR-Franchise'

In [114]:
df_delivered_awb.isna().sum()

num_rider_id     0
awb              0
trip_id          0
dispatch_time    0
node_name        0
shipping_city    0
dtype: int64

In [115]:
df_delivered_awb = df_delivered_awb.dropna()

In [116]:
df_delivered_awb['num_rider_id']=df_delivered_awb['num_rider_id'].astype(int)

In [117]:
df_delivered_city = df_delivered_awb.groupby('shipping_city').aggregate(
    {'awb':'count'
     }).reset_index()
df_delivered_city.head()

,shipping_city,awb
0,Bangalore,152481
1,Delhi,104752
2,Hyderabad,75811
3,Jaipur,8611
4,Mumbai,74907


In [118]:
df_delivered_rider = df_delivered_awb.groupby('num_rider_id').aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
df_delivered_rider.head()

,num_rider_id,orders
0,840,1545
1,845,2017
2,850,1777
3,1000,1426
4,1014,1780


In [119]:
df_delivered_franchise = df_delivered_awb.groupby(['num_rider_id','node_name']).aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
# df_delivered_node_filtered = df_delivered_franchise[df_delivered_franchise['node_name']!='BLR_OFSC']
# df_delivered_node_filtered.head()
df_delivered_franchise.head()

,num_rider_id,node_name,orders
0,840,DS BLR MTH,1545
1,845,DS BLR BOMM,2017
2,850,DS BLR MTH,1777
3,1000,BLDR-Franchise,1426
4,1014,DS BLR BOMM,1780


In [120]:
df_delivered_node = df_delivered_awb.groupby(['node_name']).aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
df_delivered_node_filtered = df_delivered_node[df_delivered_node['node_name']!='BLR_OFSC']
df_delivered_node_filtered.head()
df_delivered_node_filtered.head()

,node_name,orders
0,BLDR-Franchise,10822
1,BLR FRH ECTY,16667
2,BRBN-Franchise,5005
3,CMRJ-Franchise,6839
4,DEL FRH GKP,10303


In [121]:
df_delivered_node_filtered

,node_name,orders
0,BLDR-Franchise,10822
1,BLR FRH ECTY,16667
2,BRBN-Franchise,5005
3,CMRJ-Franchise,6839
4,DEL FRH GKP,10303
5,DS BLR BOMM,33862
6,DS BLR HBBL,30806
7,DS BLR MTH,31737
8,DS BOM KDWL,1097
9,DS BOM KRL,36617


##### Rider Pay CPO

In [122]:
df_payout = df_rider_cost_filtered.merge(df_delivered_rider, left_on='rider_id',right_on='num_rider_id', how='left').drop(columns=['num_rider_id'])
df_payout.head()

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,exist,orders
0,Bangalore,3043,A SANTHOSH,DS BLR MTH,Peel works,ORDER,51010.0,0,0.0,0.0,0,51010.0,2024-03-01,Bangalore-45352-Peel works,1.0,1667.0
1,Bangalore,3587,Abdul manaf,DS BLR BOMM,Hindusree,ORDER,46750.0,0,0.0,0.0,0,46750.0,2024-03-01,Bangalore-45352-Hindusree,1.0,1525.0
2,Bangalore,2978,Abhishek MS,DS BLR BOMM,Peel works,FIXED,20320.0,0,0.0,0.0,0,20320.0,2024-03-01,Bangalore-45352-Peel works,1.0,386.0
3,Bangalore,1930,Aejaz pasha,DS BLR BOMM,Quess Corp,ORDER,42490.0,0,0.0,0.0,0,42490.0,2024-03-01,Bangalore-45352-Quess Corp,1.0,1383.0
4,Bangalore,4300,Ananda M,DS BLR BOMM,Hindusree,FIXED,17250.0,0,2398.0,0.0,2398,14852.0,2024-03-01,Bangalore-45352-Hindusree,1.0,322.0


In [123]:
df_payout['rider_pay_cpo'] = df_payout['total_pay'].astype(float)/df_payout['orders']
df_payout.head()

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,exist,orders,rider_pay_cpo
0,Bangalore,3043,A SANTHOSH,DS BLR MTH,Peel works,ORDER,51010.0,0,0.0,0.0,0,51010.0,2024-03-01,Bangalore-45352-Peel works,1.0,1667.0,30.599880
1,Bangalore,3587,Abdul manaf,DS BLR BOMM,Hindusree,ORDER,46750.0,0,0.0,0.0,0,46750.0,2024-03-01,Bangalore-45352-Hindusree,1.0,1525.0,30.655738
2,Bangalore,2978,Abhishek MS,DS BLR BOMM,Peel works,FIXED,20320.0,0,0.0,0.0,0,20320.0,2024-03-01,Bangalore-45352-Peel works,1.0,386.0,52.642487
3,Bangalore,1930,Aejaz pasha,DS BLR BOMM,Quess Corp,ORDER,42490.0,0,0.0,0.0,0,42490.0,2024-03-01,Bangalore-45352-Quess Corp,1.0,1383.0,30.723066
4,Bangalore,4300,Ananda M,DS BLR BOMM,Hindusree,FIXED,17250.0,0,2398.0,0.0,2398,14852.0,2024-03-01,Bangalore-45352-Hindusree,1.0,322.0,53.571429


In [124]:
df_payout[df_payout['rider_id']==4342]

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,exist,orders,rider_pay_cpo
79,Bangalore,4342,Syed sala uddin,DS BLR HBBL,Peel works,ORDER,20000.0,0,0.0,0.0,0,20000.0,2024-03-01,NaN,NaN,1230.0,16.260163


##### Franchise Pay CPO

In [125]:
df_franchise_cost_filtered.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
25,2024-03-01,Bangalore,BLDR-Franchise,"405,750",0,999,0,999,"404,751"
26,2024-03-01,Bangalore,BLR FRH ECTY,"621,270",0,578,0,578,"620,692"
27,2024-03-01,Bangalore,CMRJ-Franchise,"271,202",0,"2,547",0,"2,547","268,655"
28,2024-03-01,Bangalore,STNG-Franchise,"177,328",0,568,0,568,"176,760"
29,2024-03-01,Bangalore,UUTR-Franchise,"395,844","47,031","4,492",0,"51,523","344,321"


In [126]:
df_delivered_node_filtered.head()

,node_name,orders
0,BLDR-Franchise,10822
1,BLR FRH ECTY,16667
2,BRBN-Franchise,5005
3,CMRJ-Franchise,6839
4,DEL FRH GKP,10303


In [127]:
df_franchise_payout=df_franchise_cost_filtered.merge(df_delivered_node_filtered,right_on='node_name',left_on='hub',how='left')[['node_name','orders','total_pay']]
df_franchise_payout['vendor_name'] = df_franchise_payout['node_name']
df_franchise_payout['payout_structure'] = 'ORDER'
df_franchise_payout['rider_pay_cpo'] = df_franchise_payout['total_pay'].str.replace(',', '').astype(float) / df_franchise_payout['orders']
df_franchise_payout['Key'] = df_franchise_payout['node_name']

df_franchise_payout.head()

,node_name,orders,total_pay,vendor_name,payout_structure,rider_pay_cpo,Key
0,BLDR-Franchise,10822,"405,750",BLDR-Franchise,ORDER,37.493070,BLDR-Franchise
1,BLR FRH ECTY,16667,"621,270",BLR FRH ECTY,ORDER,37.275454,BLR FRH ECTY
2,CMRJ-Franchise,6839,"271,202",CMRJ-Franchise,ORDER,39.655213,CMRJ-Franchise
3,STNG-Franchise,4496,"177,328",STNG-Franchise,ORDER,39.441281,STNG-Franchise
4,UUTR-Franchise,9935,"395,844",UUTR-Franchise,ORDER,39.843382,UUTR-Franchise


##### combining both and making rider pay

In [128]:
df_cost_master = df_delivered_awb.merge(df_payout,left_on = 'num_rider_id',right_on = 'rider_id',how='left')[['num_rider_id','rider_id','awb','trip_id','node_name','shipping_city','vendor_name','payout_structure','rider_pay_cpo','Key','dispatch_time']]
df_cost_master.head()

,num_rider_id,rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
0,4359,4359.0,BZPPL00072751,3307370,DS HYD SRNG,Hyderabad,Peel works,FIXED,39.815891,Hyderabad-45352-Peel works,2024-03-16 04:23:13.732
1,2173,2173.0,BZPPL00076974,3083035,DS HYD KMP,Hyderabad,ACP Facility,ORDER,31.906780,Hyderabad-45352-ACP Facility,2024-03-03 03:45:24.106
2,2414,2414.0,BZPPL00078356,3174745,DS BLR MTH,Bangalore,Hindusree,ORDER,30.000000,Bangalore-45352-Hindusree,2024-03-08 06:30:16.060
3,850,850.0,BZPPL00089116,3198179,DS BLR MTH,Bangalore,Peel works,ORDER,30.562746,Bangalore-45352-Peel works,2024-03-10 04:43:27.730
4,3975,NaN,BZPPL00089208,3253485,UUTR-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-03-13 05:09:10.080


In [129]:
duplicate_awb = df_payout[df_payout.duplicated('rider_id', keep=False)]
print(duplicate_awb.shape)
print(duplicate_awb)

(0, 17)
Empty DataFrame
Columns: [city, rider_id, rider_name, hub, vendor_name, payout_structure, total_pay, cod_deduction, lost_deduction, fake_deduction, total_deduction, net_pay, date, Key, exist, orders, rider_pay_cpo]
Index: []


In [130]:
# df_payout[df_payout['rider_id']==2086]

In [131]:
duplicate_awb = df_cost_master[df_cost_master.duplicated('awb', keep=False)]
print(duplicate_awb.shape)
print(duplicate_awb)

(0, 11)
Empty DataFrame
Columns: [num_rider_id, rider_id, awb, trip_id, node_name, shipping_city, vendor_name, payout_structure, rider_pay_cpo, Key, dispatch_time]
Index: []


In [132]:
df_cost_master_final = df_cost_master.merge(df_franchise_payout, on='node_name', how='left', suffixes=('', '_fill')).drop(columns=['orders','rider_id','total_pay'])

# Fill the NaN values in df_cost_master with the corresponding values from df_fill
columns_to_fill = ['vendor_name', 'payout_structure', 'rider_pay_cpo', 'Key']
for column in columns_to_fill:
    df_cost_master_final[column] = df_cost_master_final[column].fillna(df_cost_master_final[f"{column}_fill"])
df_cost_master_final = df_cost_master_final.drop(columns=[f"{column}_fill" for column in columns_to_fill])

df_cost_master_final.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
0,4359,BZPPL00072751,3307370,DS HYD SRNG,Hyderabad,Peel works,FIXED,39.815891,Hyderabad-45352-Peel works,2024-03-16 04:23:13.732
1,2173,BZPPL00076974,3083035,DS HYD KMP,Hyderabad,ACP Facility,ORDER,31.906780,Hyderabad-45352-ACP Facility,2024-03-03 03:45:24.106
2,2414,BZPPL00078356,3174745,DS BLR MTH,Bangalore,Hindusree,ORDER,30.000000,Bangalore-45352-Hindusree,2024-03-08 06:30:16.060
3,850,BZPPL00089116,3198179,DS BLR MTH,Bangalore,Peel works,ORDER,30.562746,Bangalore-45352-Peel works,2024-03-10 04:43:27.730
4,3975,BZPPL00089208,3253485,UUTR-Franchise,Bangalore,UUTR-Franchise,ORDER,39.843382,UUTR-Franchise,2024-03-13 05:09:10.080


In [133]:
df_cost_master_final.isna().sum()

num_rider_id           0
awb                    0
trip_id                0
node_name              0
shipping_city          0
vendor_name           17
payout_structure      17
rider_pay_cpo         17
Key                 1247
dispatch_time          0
dtype: int64

In [134]:
df_cost_master_final[df_cost_master_final['Key'].isna()].groupby(['num_rider_id','node_name']).aggregate({'awb':'count'})

,,awb
num_rider_id,node_name,
2716,GSCAN TEST,14
3359,GSCAN TEST,3
4342,DS BLR HBBL,1230


In [135]:
df_cost_master_final[df_cost_master_final['num_rider_id']==4342]

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
198,4342,BZPPL00151039,3091112,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-03 06:20:46.268
471,4342,BZPPL00153992,3061413,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-01 06:21:11.432
583,4342,BZPPL00154589,3091081,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-03 06:21:11.427
1324,4342,BZPPL00156573,3091099,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-03 06:21:14.157
1359,4342,BZPPL00156635,3107059,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-04 05:51:34.780
...,...,...,...,...,...,...,...,...,...,...
414957,4342,GS2142149518,3096943,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-03 12:29:21.024
415366,4342,GS2143550626,3091088,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-03 06:22:02.889
415383,4342,GS2143607412,3125627,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-05 06:56:31.592
415961,4342,GS2145619689,3425660,DS BLR HBBL,Bangalore,Peel works,ORDER,16.260163,NaN,2024-03-22 06:08:27.871


In [136]:
missing = df_cost_master_final[df_cost_master_final['node_name'].isna()].groupby(['num_rider_id','node_name']).aggregate({'awb':'count'}).reset_index()
missing[missing['node_name'] ==  'DS HYD SRNG']['num_rider_id']
# missing

Series([], Name: num_rider_id, dtype: int64)

In [137]:
df_cost_master_final.dropna(inplace=True)

In [138]:
duplicate_awb = df_cost_master_final[df_cost_master_final.duplicated('awb', keep=False)]
print(duplicate_awb.shape)
print(duplicate_awb)

(0, 10)
Empty DataFrame
Columns: [num_rider_id, awb, trip_id, node_name, shipping_city, vendor_name, payout_structure, rider_pay_cpo, Key, dispatch_time]
Index: []


##### Vendor Pay CPO

In [139]:
df_vendor_cost_filtered = df_vendor_cost[['Key','Extra per rs']]
df_cost_master2 = df_cost_master_final.merge(df_vendor_cost_filtered,on='Key',how='left')
df_cost_master2.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time,Extra per rs
0,4359,BZPPL00072751,3307370,DS HYD SRNG,Hyderabad,Peel works,FIXED,39.815891,Hyderabad-45352-Peel works,2024-03-16 04:23:13.732,0.278767
1,2173,BZPPL00076974,3083035,DS HYD KMP,Hyderabad,ACP Facility,ORDER,31.906780,Hyderabad-45352-ACP Facility,2024-03-03 03:45:24.106,0.410541
2,2414,BZPPL00078356,3174745,DS BLR MTH,Bangalore,Hindusree,ORDER,30.000000,Bangalore-45352-Hindusree,2024-03-08 06:30:16.060,0.048429
3,850,BZPPL00089116,3198179,DS BLR MTH,Bangalore,Peel works,ORDER,30.562746,Bangalore-45352-Peel works,2024-03-10 04:43:27.730,0.174223
4,3975,BZPPL00089208,3253485,UUTR-Franchise,Bangalore,UUTR-Franchise,ORDER,39.843382,UUTR-Franchise,2024-03-13 05:09:10.080,NaN


In [140]:
df_cost_master2['vendor_cpo']=df_cost_master2['rider_pay_cpo']*df_cost_master2['Extra per rs']
df_cost_master2.drop(columns=['Extra per rs','Key'],inplace=True)
df_cost_master2.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo
0,4359,BZPPL00072751,3307370,DS HYD SRNG,Hyderabad,Peel works,FIXED,39.815891,2024-03-16 04:23:13.732,11.099338
1,2173,BZPPL00076974,3083035,DS HYD KMP,Hyderabad,ACP Facility,ORDER,31.906780,2024-03-03 03:45:24.106,13.099043
2,2414,BZPPL00078356,3174745,DS BLR MTH,Bangalore,Hindusree,ORDER,30.000000,2024-03-08 06:30:16.060,1.452882
3,850,BZPPL00089116,3198179,DS BLR MTH,Bangalore,Peel works,ORDER,30.562746,2024-03-10 04:43:27.730,5.324735
4,3975,BZPPL00089208,3253485,UUTR-Franchise,Bangalore,UUTR-Franchise,ORDER,39.843382,2024-03-13 05:09:10.080,NaN


In [141]:
df_cost_master2.isna().sum()

num_rider_id             0
awb                      0
trip_id                  0
node_name                0
shipping_city            0
vendor_name              0
payout_structure         0
rider_pay_cpo            0
dispatch_time            0
vendor_cpo          127720
dtype: int64

In [142]:
df_cost_master2=df_cost_master2.fillna('0')

In [143]:
df_cost_master2.isna().sum()

num_rider_id        0
awb                 0
trip_id             0
node_name           0
shipping_city       0
vendor_name         0
payout_structure    0
rider_pay_cpo       0
dispatch_time       0
vendor_cpo          0
dtype: int64

In [144]:
duplicate_awb = df_cost_master2[df_cost_master2.duplicated('awb', keep=False)]
print(duplicate_awb)

Empty DataFrame
Columns: [num_rider_id, awb, trip_id, node_name, shipping_city, vendor_name, payout_structure, rider_pay_cpo, dispatch_time, vendor_cpo]
Index: []


##### Fuel CPO

In [145]:

df_fuel_pay = df_delivered_city.merge(df_fuel_cost_grouped,left_on='shipping_city',right_on='city',how='left')
df_fuel_pay.head()

,shipping_city,awb,city,amount
0,Bangalore,152481,Bangalore,547689
1,Delhi,104752,Delhi,414632
2,Hyderabad,75811,Hyderabad,477601
3,Jaipur,8611,Jaipur,105902
4,Mumbai,74907,Mumbai,509398


In [146]:
df_fuel_pay['fuel_cpo']=df_fuel_pay['amount']/df_fuel_pay['awb']
df_fuel_pay.drop(columns=['amount','awb','city'],inplace=True)

In [147]:
df_cost_master3 = df_cost_master2.merge(df_fuel_pay,on='shipping_city',how='left')
df_cost_master3.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,4359,BZPPL00072751,3307370,DS HYD SRNG,Hyderabad,Peel works,FIXED,39.815891,2024-03-16 04:23:13.732,11.099338,6.299891
1,2173,BZPPL00076974,3083035,DS HYD KMP,Hyderabad,ACP Facility,ORDER,31.906780,2024-03-03 03:45:24.106,13.099043,6.299891
2,2414,BZPPL00078356,3174745,DS BLR MTH,Bangalore,Hindusree,ORDER,30.000000,2024-03-08 06:30:16.060,1.452882,3.591851
3,850,BZPPL00089116,3198179,DS BLR MTH,Bangalore,Peel works,ORDER,30.562746,2024-03-10 04:43:27.730,5.324735,3.591851
4,3975,BZPPL00089208,3253485,UUTR-Franchise,Bangalore,UUTR-Franchise,ORDER,39.843382,2024-03-13 05:09:10.080,0,3.591851


In [148]:
df_cost_master3.isna().sum()

num_rider_id        0
awb                 0
trip_id             0
node_name           0
shipping_city       0
vendor_name         0
payout_structure    0
rider_pay_cpo       0
dispatch_time       0
vendor_cpo          0
fuel_cpo            0
dtype: int64

In [149]:
df_cost_master3=df_cost_master3.fillna(0)

In [150]:
df_cost_master3.shape

(415315, 11)

##### Merging with revenue

Note we need to attribute fm-mm cost to RTO Delivered also, so we need to get RTO orders from revenue

In [151]:
df_revenue_filtered.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
0,GS1205975883,Decathlon,Bangalore,560100,Delivered,Prepaid,60.0,54.05,301,Bangalore
1,GS2131162508,Shiprocket NDD,Bangalore,560099,RTO Delivered,COD,300.0,34.50,301,Bangalore
2,GS1123757704,Shiprocket Intercity,Bangalore,500089,Delivered,Prepaid,392.0,31.00,301,Bangalore
3,GS1443308699,adidas,Delhi,110045,Delivered,Prepaid,500.0,21.00,301,Delhi
4,GS1788247667,Bebodywise,Bangalore,560011,RTO Delivered,COD,180.0,36.00,301,Bangalore


In [152]:
df_cost_master3.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,4359,BZPPL00072751,3307370,DS HYD SRNG,Hyderabad,Peel works,FIXED,39.815891,2024-03-16 04:23:13.732,11.099338,6.299891
1,2173,BZPPL00076974,3083035,DS HYD KMP,Hyderabad,ACP Facility,ORDER,31.906780,2024-03-03 03:45:24.106,13.099043,6.299891
2,2414,BZPPL00078356,3174745,DS BLR MTH,Bangalore,Hindusree,ORDER,30.000000,2024-03-08 06:30:16.060,1.452882,3.591851
3,850,BZPPL00089116,3198179,DS BLR MTH,Bangalore,Peel works,ORDER,30.562746,2024-03-10 04:43:27.730,5.324735,3.591851
4,3975,BZPPL00089208,3253485,UUTR-Franchise,Bangalore,UUTR-Franchise,ORDER,39.843382,2024-03-13 05:09:10.080,0,3.591851


In [153]:
duplicate_awb = df_cost_master3[df_cost_master3.duplicated('awb', keep=False)]
print(duplicate_awb)

Empty DataFrame
Columns: [num_rider_id, awb, trip_id, node_name, shipping_city, vendor_name, payout_structure, rider_pay_cpo, dispatch_time, vendor_cpo, fuel_cpo]
Index: []


In [154]:
# df_margin = df_revenue_filtered.merge(df_cost_master3,on='awb',how='outer')
df_margin = df_revenue_filtered.merge(df_cost_master3,on='awb',how='outer').drop(columns=['shipping_city_y']).rename(columns={'shipping_city_x':'shipping_city'})
df_margin.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city,num_rider_id,trip_id,node_name,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,BZPPL00072751,Purplle,Hyderabad,500055,Delivered,COD,1830.0,35.0,301.0,Hyderabad,4359.0,3307370.0,DS HYD SRNG,Peel works,FIXED,39.815891,2024-03-16 04:23:13.732,11.099338,6.299891
1,BZPPL00074613,Purplle,Hyderabad,500055,RTO Delivered,COD,455.0,35.0,301.0,Hyderabad,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,BZPPL00076974,Purplle,Hyderabad,500090,Delivered,COD,460.0,35.0,301.0,Hyderabad,2173.0,3083035.0,DS HYD KMP,ACP Facility,ORDER,31.906780,2024-03-03 03:45:24.106,13.099043,6.299891
3,BZPPL00078356,Purplle,Bangalore,560008,Delivered,COD,924.0,35.0,301.0,Bangalore,2414.0,3174745.0,DS BLR MTH,Hindusree,ORDER,30.000000,2024-03-08 06:30:16.060,1.452882,3.591851
4,BZPPL00089116,Purplle,Bangalore,560048,Delivered,Prepaid,436.8,35.0,301.0,Bangalore,850.0,3198179.0,DS BLR MTH,Peel works,ORDER,30.562746,2024-03-10 04:43:27.730,5.324735,3.591851


In [155]:
df_margin.shape

(446143, 19)

In [156]:
duplicate_awb = df_margin[df_margin.duplicated('awb', keep=False)]
print(duplicate_awb)

Empty DataFrame
Columns: [awb, user_name, warehouse_city, drop_pincode, shipment_status, mop, chargeable_weight, shipment_charges, shipper_id, shipping_city, num_rider_id, trip_id, node_name, vendor_name, payout_structure, rider_pay_cpo, dispatch_time, vendor_cpo, fuel_cpo]
Index: []


In [157]:
duplicate_awb.shape

(0, 19)

In [158]:
columns_to_update = [
    'num_rider_id', 'trip_id', 'node_name', 'vendor_name', 
    'payout_structure', 'rider_pay_cpo', 'dispatch_time', 
    'vendor_cpo', 'fuel_cpo'
]
df_margin.loc[df_margin['shipment_status'] == 'RTO Delivered', columns_to_update] = 0

In [159]:
df_margin.isna().sum()

awb                     0
user_name               7
warehouse_city          7
drop_pincode            7
shipment_status         7
mop                     7
chargeable_weight       7
shipment_charges        7
shipper_id              7
shipping_city           7
num_rider_id         1237
trip_id              1237
node_name            1237
vendor_name          1237
payout_structure     1237
rider_pay_cpo        1237
dispatch_time        1237
vendor_cpo           1237
fuel_cpo             1237
dtype: int64

In [160]:
df_margin = df_margin.dropna()

##### FM-MM Costing

In [161]:
df_mm_cost_filtered.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
18,01/03/2024,Delhi,Bangalore,Rippler,626997.0,NaN,Intercity
19,01/03/2024,Bangalore,Bangalore,NaN,1657356.0,NaN,Hyperlocal
20,01/03/2024,Delhi,Delhi,NaN,1412767.0,NaN,Hyperlocal
21,01/03/2024,Delhi,Hyderabad,Rippler,334523.0,NaN,Intercity
22,01/03/2024,Bangalore,Hyderabad,Rippler,665012.0,NaN,Intercity


In [162]:
df_mm_cost_filtered['Cost'].sum()

np.float64(6114598.0)

In [163]:
mm_pay=df_mm_cost_filtered.groupby('Shipping City').aggregate({'Cost':'sum'}).reset_index()
mm_pay

,Shipping City,Cost
0,Bangalore,2284353.0
1,Delhi,1412767.0
2,Hyderabad,1359687.0
3,Jaipur,153617.0
4,Mumbai,904174.0


In [164]:
##### Removing 3WH FROM FM-MM
three_wh_pay=df_rider_cost_filtered[df_rider_cost_filtered['vendor_name'] == '3WH'].groupby('city').aggregate({'total_pay':'sum'}).reset_index()
mm_pay = mm_pay.merge(three_wh_pay,left_on='Shipping City',right_on='city',how='left').drop(columns=['city']).fillna(0)
mm_pay['Cost'] = mm_pay['Cost']-mm_pay['total_pay'].astype(float)
mm_pay.drop(columns=['total_pay'],inplace=True)


##### starting rto delivered fm

In [165]:
mm_payout_rto = mm_pay.merge(df_rto_per,right_on='shipping_city',left_on='Shipping City',how='left')
mm_payout_rto.head()

,Shipping City,Cost,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per
0,Bangalore,2247023.0,Bangalore,Delivered,183543.0,RTO Delivered,9177.0,0.047618
1,Delhi,1412767.0,Delhi,Delivered,127657.0,RTO Delivered,12576.0,0.089679
2,Hyderabad,1359687.0,Hyderabad,Delivered,34688.0,RTO Delivered,2365.0,0.063827
3,Jaipur,153617.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Mumbai,904174.0,Mumbai,Delivered,70653.0,RTO Delivered,5477.0,0.071943


In [166]:
mm_payout_rto['r_factor']=mm_payout_rto['Cost']/(mm_payout_rto['orders_x']+2*mm_payout_rto['orders_y']).astype(float)
mm_payout_rto['r_factor'] = mm_payout_rto['r_factor'].round(2)
mm_payout_rto.head()

,Shipping City,Cost,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per,r_factor
0,Bangalore,2247023.0,Bangalore,Delivered,183543.0,RTO Delivered,9177.0,0.047618,11.13
1,Delhi,1412767.0,Delhi,Delivered,127657.0,RTO Delivered,12576.0,0.089679,9.25
2,Hyderabad,1359687.0,Hyderabad,Delivered,34688.0,RTO Delivered,2365.0,0.063827,34.49
3,Jaipur,153617.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mumbai,904174.0,Mumbai,Delivered,70653.0,RTO Delivered,5477.0,0.071943,11.08


In [167]:
margin2 = df_margin.merge(mm_payout_rto[['shipping_city','r_factor']], on='shipping_city',how='left').reset_index()

In [168]:
margin2['fm_mm_cpo'] = np.where(
    margin2['shipment_status'] == 'RTO Delivered',
    2 * margin2['r_factor'],
    np.where(
        margin2['shipment_status'] == 'Delivered',
        margin2['r_factor'],
        0  # Or any default value you prefer
    )
)
margin2.head()

,index,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,...,trip_id,node_name,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo,r_factor,fm_mm_cpo
0,0,BZPPL00072751,Purplle,Hyderabad,500055,Delivered,COD,1830.0,35.0,301.0,...,3307370.0,DS HYD SRNG,Peel works,FIXED,39.815891,2024-03-16 04:23:13.732000,11.099338,6.299891,34.49,34.49
1,1,BZPPL00074613,Purplle,Hyderabad,500055,RTO Delivered,COD,455.0,35.0,301.0,...,0.0,0,0,0,0.000000,0,0,0.000000,34.49,68.98
2,2,BZPPL00076974,Purplle,Hyderabad,500090,Delivered,COD,460.0,35.0,301.0,...,3083035.0,DS HYD KMP,ACP Facility,ORDER,31.906780,2024-03-03 03:45:24.106000,13.099043,6.299891,34.49,34.49
3,3,BZPPL00078356,Purplle,Bangalore,560008,Delivered,COD,924.0,35.0,301.0,...,3174745.0,DS BLR MTH,Hindusree,ORDER,30.000000,2024-03-08 06:30:16.060000,1.452882,3.591851,11.13,11.13
4,4,BZPPL00089116,Purplle,Bangalore,560048,Delivered,Prepaid,436.8,35.0,301.0,...,3198179.0,DS BLR MTH,Peel works,ORDER,30.562746,2024-03-10 04:43:27.730000,5.324735,3.591851,11.13,11.13


In [169]:
margin2[margin2['shipment_status']=='RTO Delivered']['vendor_name'] = 'RTO Delivered'

In [170]:
margin2.isna().sum()

index                0
awb                  0
user_name            0
warehouse_city       0
drop_pincode         0
shipment_status      0
mop                  0
chargeable_weight    0
shipment_charges     0
shipper_id           0
shipping_city        0
num_rider_id         0
trip_id              0
node_name            0
vendor_name          0
payout_structure     0
rider_pay_cpo        0
dispatch_time        0
vendor_cpo           0
fuel_cpo             0
r_factor             0
fm_mm_cpo            0
dtype: int64

In [171]:
margin3 = margin2.drop(columns=['index','user_name','chargeable_weight','shipper_id','r_factor'])
margin3.rename(columns={
    'shipment_charges':'rpo','payout_structure':'rider_payout_structure','fuel_cpo':'rider_fuel_cpo'
},inplace = True)
margin3.head()

,awb,warehouse_city,drop_pincode,shipment_status,mop,rpo,shipping_city,num_rider_id,trip_id,node_name,vendor_name,rider_payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,rider_fuel_cpo,fm_mm_cpo
0,BZPPL00072751,Hyderabad,500055,Delivered,COD,35.0,Hyderabad,4359.0,3307370.0,DS HYD SRNG,Peel works,FIXED,39.815891,2024-03-16 04:23:13.732000,11.099338,6.299891,34.49
1,BZPPL00074613,Hyderabad,500055,RTO Delivered,COD,35.0,Hyderabad,0.0,0.0,0,0,0,0.000000,0,0,0.000000,68.98
2,BZPPL00076974,Hyderabad,500090,Delivered,COD,35.0,Hyderabad,2173.0,3083035.0,DS HYD KMP,ACP Facility,ORDER,31.906780,2024-03-03 03:45:24.106000,13.099043,6.299891,34.49
3,BZPPL00078356,Bangalore,560008,Delivered,COD,35.0,Bangalore,2414.0,3174745.0,DS BLR MTH,Hindusree,ORDER,30.000000,2024-03-08 06:30:16.060000,1.452882,3.591851,11.13
4,BZPPL00089116,Bangalore,560048,Delivered,Prepaid,35.0,Bangalore,850.0,3198179.0,DS BLR MTH,Peel works,ORDER,30.562746,2024-03-10 04:43:27.730000,5.324735,3.591851,11.13


In [172]:
margin3.to_csv('margin_march.csv')

##### Vendor Level Margin

In [173]:
vendor_margins=margin3.groupby(['shipping_city','vendor_name','num_rider_id']).aggregate({
    'awb': 'count',
    'rider_pay_cpo': 'sum',
    'vendor_cpo': 'sum',
    'rider_fuel_cpo': 'sum',
    'fm_mm_cpo': 'sum',
    'rpo':'sum'
}).reset_index()
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay_cpo,vendor_cpo,rider_fuel_cpo,fm_mm_cpo,rpo
0,Bangalore,0,0.0,9177,0.000000,0,0.000000,204280.02,722929.429700
1,Bangalore,3WH,4118.0,106,4505.000000,0.0,380.736184,1179.78,8802.906075
2,Bangalore,3WH,4309.0,89,2670.000000,0.0,319.674720,990.57,8854.254775
3,Bangalore,3WH,4673.0,438,20192.160494,0.0,1573.230645,4874.94,46240.697000
4,Bangalore,3WH,5204.0,204,6120.000000,0.0,732.737561,2270.52,20898.824000


In [174]:
vendor_margins.dtypes

shipping_city      object
vendor_name        object
num_rider_id      float64
awb                 int64
rider_pay_cpo     float64
vendor_cpo         object
rider_fuel_cpo    float64
fm_mm_cpo         float64
rpo               float64
dtype: object

In [175]:
vendor_margins['vendor_cpo'] = vendor_margins['vendor_cpo'].astype(float)
vendor_margins['fm_mm_cpo'] = vendor_margins['fm_mm_cpo'].astype(float)

In [176]:

vendor_margins['total_cost']=vendor_margins['rider_pay_cpo']+vendor_margins['vendor_cpo']+vendor_margins['rider_fuel_cpo']+vendor_margins['fm_mm_cpo']
vendor_margins['margin']=vendor_margins['rpo']-vendor_margins['total_cost']
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay_cpo,vendor_cpo,rider_fuel_cpo,fm_mm_cpo,rpo,total_cost,margin
0,Bangalore,0,0.0,9177,0.000000,0.0,0.000000,204280.02,722929.429700,204280.020000,518649.409700
1,Bangalore,3WH,4118.0,106,4505.000000,0.0,380.736184,1179.78,8802.906075,6065.516184,2737.389891
2,Bangalore,3WH,4309.0,89,2670.000000,0.0,319.674720,990.57,8854.254775,3980.244720,4874.010055
3,Bangalore,3WH,4673.0,438,20192.160494,0.0,1573.230645,4874.94,46240.697000,26640.331139,19600.365861
4,Bangalore,3WH,5204.0,204,6120.000000,0.0,732.737561,2270.52,20898.824000,9123.257561,11775.566439


In [177]:
vendor_margins.rename(columns={'fuel_cpo':'fuel_pay','mm_cpo':'mm_pay','vendor_cpo':'vendor_pay','rider_pay_cpo':'rider_pay'},inplace=True)

vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay,vendor_pay,rider_fuel_cpo,fm_mm_cpo,rpo,total_cost,margin
0,Bangalore,0,0.0,9177,0.000000,0.0,0.000000,204280.02,722929.429700,204280.020000,518649.409700
1,Bangalore,3WH,4118.0,106,4505.000000,0.0,380.736184,1179.78,8802.906075,6065.516184,2737.389891
2,Bangalore,3WH,4309.0,89,2670.000000,0.0,319.674720,990.57,8854.254775,3980.244720,4874.010055
3,Bangalore,3WH,4673.0,438,20192.160494,0.0,1573.230645,4874.94,46240.697000,26640.331139,19600.365861
4,Bangalore,3WH,5204.0,204,6120.000000,0.0,732.737561,2270.52,20898.824000,9123.257561,11775.566439


In [178]:
vendor_margins.to_csv('vendor_margins_march.csv')

In [179]:
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay,vendor_pay,rider_fuel_cpo,fm_mm_cpo,rpo,total_cost,margin
0,Bangalore,0,0.0,9177,0.000000,0.0,0.000000,204280.02,722929.429700,204280.020000,518649.409700
1,Bangalore,3WH,4118.0,106,4505.000000,0.0,380.736184,1179.78,8802.906075,6065.516184,2737.389891
2,Bangalore,3WH,4309.0,89,2670.000000,0.0,319.674720,990.57,8854.254775,3980.244720,4874.010055
3,Bangalore,3WH,4673.0,438,20192.160494,0.0,1573.230645,4874.94,46240.697000,26640.331139,19600.365861
4,Bangalore,3WH,5204.0,204,6120.000000,0.0,732.737561,2270.52,20898.824000,9123.257561,11775.566439


In [180]:
vendor_margins['awb'].sum()

np.int64(444899)

##### Hub Losss Total

In [181]:
df_hubloss_cost_filtered = df_hubloss_cost[df_hubloss_cost['Month']=='01/03/2024']
df_hubloss_cost_filtered['Amount']=df_hubloss_cost_filtered['Amount'].str.replace(',','').astype(float)
df_hubloss_cost_filtered

,Head,Team,wh_code,Amount,Month
8,Loss on Hub,Bangalore,DS_BLR_MTH,13827.00,01/03/2024
9,Loss on Hub,Bangalore,DS_BLR_BOMM,1099.00,01/03/2024
10,Loss on Hub,Mumbai,DS_BOM_KDWL,10650.00,01/03/2024
11,Loss on Hub,Mumbai,DS_BOM_KRL,22694.00,01/03/2024
12,Loss on Hub,Mumbai,DS_BOM_SWRI,5650.00,01/03/2024
13,Loss on Hub,Mumbai,DS_BOM_THAN,1928.00,01/03/2024
14,Loss on Hub,Hyderabad,DS_HYD_KMP,12640.00,01/03/2024
15,Loss on Hub,Hyderabad,DS_HYD_MHDP,10061.50,01/03/2024
16,Loss on Hub,Hyderabad,DS_HYD_SRNG,122073.54,01/03/2024
17,Loss on Hub,Delhi,DS_NCR_DWK,48674.00,01/03/2024


In [182]:
df_hubloss_cost_filtered.groupby('Team').aggregate({
    'Amount':'sum'
}).reset_index()

,Team,Amount
0,Bangalore,14926.00
1,Delhi,50022.00
2,Hyderabad,144775.04
3,Mumbai,40922.00
